In [83]:
import pandas as pd
import numpy as np
import pymysql
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [84]:
def convertir_a_meses(periodo):
    # Dividimos la cadena en partes
    partes = periodo.split()
    
    # Inicializamos los años y meses a 0
    años = 0
    meses = 0
    
    # Iteramos sobre las partes y extraemos los valores de años y meses
    for i in range(len(partes)):
        if partes[i] == "AÑOS":
            años = int(partes[i - 1])
        elif partes[i] == "MESES":
            meses = int(partes[i - 1])
    
    # Convertimos todo a meses
    total_meses = años * 12 + meses
    return total_meses

In [85]:
def excel_serial_to_date(serial):
    # La fecha base de Excel es el 1 de enero de 1900
    excel_base_date = datetime(1899, 12, 30)
    # Sumar el número de serie a la fecha base
    converted_date = excel_base_date + timedelta(days=serial)
    return converted_date

In [86]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')
cursor = cnx.cursor()

In [87]:
replace_dict_oficios = {'jarcieria': 'servicios',
 'tlapaleria/ferreteria/materiales de construccion': 'servicios',
 'lavanderia/tintoreria/planchaduria': 'servicios',
 'combustibles/gas/gasolina': 'servicios',
 'animales/veterinaria': 'servicios',
 'desperdicios/residuos/reciclaje': 'bienes de consumo',
 'calzado reparacion y venta': 'bienes de consumo',
 'venta por catalogo': 'ventas',
 'venta de ropa': 'ventas',
 'venta de comida': 'ventas',
 'distribuidor con experiencia': 'ventas',
 'venta de alimentos': 'venta de alimentos',
 'belleza/salon/estetica/barberia': 'estetica',
 'joyeria/bisuteria/fundicion': 'estetica',
 'eventos/salones/sillas/mesas': 'eventos',
 'flores/arreglos': 'eventos',
 'fotografia/video': 'eventos',
 'imprenta/serigrafia/rotulos': 'papeleria',
 'cafeinternet': 'papeleria',
 'papeleria': 'papeleria',
 'agua/potable/residual': 'bienes de consumo',
 'carniceria/polleria/rastro': 'bienes de consumo',
 'tortilleria/molino': 'bienes de consumo',
 'frutas/verduras/semillas/campo/agropecuarias': 'bienes de consumo',
 'abarrotes/miscelaneas/perecederos': 'bienes de consumo',
 'panaderia': 'bienes de consumo',
 'amedicina/salud/farmacia': 'salud',
 'deportes/gimnasio': 'prestamo de servicios',
 'educacion/escuelas/centro de capacitacion': 'prestamo de servicios',
 'venta de autos/motocicletas/lotes/autopartes': 'venta de electronicos/automoviles',
 'venta de celulares/tecnologia': 'venta de electronicos/automoviles',
 'venta de muebles/electronicos/electrodomesticos': 'venta de electronicos/automoviles',
 'electricista': 'oficios',
 'plomeria': 'oficios',
 'herreria': 'oficios',
 'cerrajeria': 'oficios',
 'arte/artesanias': 'oficios',
 'fabrica/taller/elaboracion/reparacion': 'oficios',
 'madera/carpinteria/ebanista': 'oficios',
 'mecanica automotriz/industrial': 'oficios'}

replace_dict_sucursal = {
    "amecameca": "Grupo 1",
    "milpa alta": "Grupo 1",
    "tlahuac": "Grupo 1",
    "coatzacoalcos": "Grupo 3",
    "fortin": "Grupo 3",
    "orizaba": "Grupo 3",
    "texcoco": "Grupo 2",
    "texcoco 2": "Grupo 2",
    "villahermosa": "Grupo 2",
    "chalco": "Grupo 2",
    "cardenas": "Grupo 2",
    "ixtapaluca": "Grupo 2",
    "iztapalapa": "Grupo 4",
    "nezahualcoyotl": "Grupo 4",
    "nezahualcoyotl 2": "Grupo 4",
    "chimalhuacan 2": "Grupo 4",
    "tonala": "Grupo 5",
    "tonala 2": "Grupo 5"
}


In [88]:
data_pc = pd.read_excel("./data/240430 PERDIDA DE COSECHA_.xlsb", sheet_name='COLOCACION', header=1)

In [89]:
data_pc.columns = [k.lower().replace(" ","_") for k in data_pc.columns]
data_pc = data_pc.query(f"tipo_producto == 'PRESTAMO PERSONAL RED'")

In [90]:
data_pc = data_pc[['id_distribuidor_linea_de_credito', 'perdida_de_cosecha', 'inicio_credito', 'ciclo_credito']]
data_pc.rename(columns={"id_distribuidor_linea_de_credito": "id_distribuidor"}, inplace=True)
data_pc.inicio_credito = data_pc.inicio_credito.apply(excel_serial_to_date)
data_pc.id_distribuidor = data_pc.id_distribuidor.astype(int)

In [91]:
data_pc = data_pc.query("inicio_credito < '2024-01-01'")

In [41]:
#data_pc = pd.read_excel("./data/240221 ANALISIS DE MODELOS.xlsx", sheet_name='Hoja1')

In [42]:
"""
data_pc.columns = [str(k).lower().replace(" ", "_") for k in data_pc.columns]

data_pc = data_pc[['id_distribuidor', 'p_cosecha']]

data_pc.head(1)
"""

'\ndata_pc.columns = [str(k).lower().replace(" ", "_") for k in data_pc.columns]\n\ndata_pc = data_pc[[\'id_distribuidor\', \'p_cosecha\']]\n\ndata_pc.head(1)\n'

In [92]:
ids = ", ".join(data_pc.id_distribuidor.unique().astype(str))

In [93]:
sql = f"""
select *  from 
	(select id,  escolaridad, lower(sucursal) as sucursal, lower(tipo_negocio) as tipo_negocio, tiempo_op_negocio, compras, 
		ingreso_mensual_negocio, ventas from cc_prospectos_demograficos
UNION        
	select id,  escolaridad, lower(sucursal) as sucursal, lower(tipo_negocio) as tipo_negocio, tiempo_op_negocio, compras, 
		ingreso_mensual as 'ingreso_mensual_negocio', ventas from demograficos_distribuidores
) as un
where un.id in ({ids})
"""
cursor.execute(sql)
data_demo = cursor.fetchall()
data_demo = pd.DataFrame(data_demo, columns=[i[0] for i in cursor.description])
data_demo.rename(columns={"id" : 'id_distribuidor'}, inplace=True)
data_demo.id_distribuidor = data_demo.id_distribuidor.astype(int)

In [94]:
data_demo.tipo_negocio = data_demo.tipo_negocio.fillna("venta de alimentos")
data_demo.compras = data_demo.compras.fillna(1748)
data_demo.ventas = data_demo.ventas.fillna(4465)

In [95]:
data_demo.isna().sum(0)

id_distribuidor            0
escolaridad                0
sucursal                   0
tipo_negocio               0
tiempo_op_negocio          0
compras                    0
ingreso_mensual_negocio    0
ventas                     0
dtype: int64

In [96]:
data_demo.tipo_negocio = data_demo.tipo_negocio.apply(lambda x: replace_dict_oficios[x].replace(" ","_").upper())
data_demo.tiempo_op_negocio = data_demo.tiempo_op_negocio.apply(convertir_a_meses)

In [97]:
data_demo.sucursal = data_demo.sucursal.apply(lambda x: replace_dict_sucursal[x].replace(" ","_").upper())

In [98]:
data_demo.ingreso_mensual_negocio = data_demo.ingreso_mensual_negocio.apply(lambda x: float(x.replace("$", "").replace(",", "")))

In [101]:
sql = f"""
select 
		nombreOtorgante as 'NOMBREOTORGANTE', 
		pc.folioConsulta as 'FOLIO', 
        importeCredito as 'IMPORTECREDITO', 
        fechaConsulta,
        idCliente as 'id_distribuidor',
        valor as 'SCORE_FICO' from cc_prospectos_consultas as pc
left join cc_prospectos_folios as pf on pc.folioConsulta = pf.folioConsulta 
left join cc_prospectos_scores as ps on pc.folioConsulta = ps.folioConsulta 
where idCliente in ({ids}) and fechaConsulta >= '2022-01-01' and fechaConsulta <= '2023-12-31'
"""

In [102]:
cursor.execute(sql)
data_cc = cursor.fetchall()
data_cc = pd.DataFrame(data_cc, columns=[i[0] for i in cursor.description])
data_cc.rename(columns={"id" : 'id_distribuidor'}, inplace = True)
data_cc.id_distribuidor = data_cc.id_distribuidor.astype(int)

In [103]:
data_enviar = data_demo.merge(data_cc, on='id_distribuidor')

In [104]:
data_enviar.escolaridad = data_enviar.escolaridad.str.upper()

In [105]:
data_enviar.columns = [k.upper() for k in data_enviar.columns]

data_enviar.rename(columns={"ID_DISTRIBUIDOR":"APPLICANT_ID"}, inplace=True)

In [106]:
data_enviar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104783 entries, 0 to 104782
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   APPLICANT_ID             104783 non-null  int32  
 1   ESCOLARIDAD              104783 non-null  object 
 2   SUCURSAL                 104783 non-null  object 
 3   TIPO_NEGOCIO             104783 non-null  object 
 4   TIEMPO_OP_NEGOCIO        104783 non-null  int64  
 5   COMPRAS                  104783 non-null  float64
 6   INGRESO_MENSUAL_NEGOCIO  104783 non-null  float64
 7   VENTAS                   104783 non-null  float64
 8   NOMBREOTORGANTE          104783 non-null  object 
 9   FOLIO                    104783 non-null  object 
 10  IMPORTECREDITO           104783 non-null  int64  
 11  FECHACONSULTA            104783 non-null  object 
 12  SCORE_FICO               104783 non-null  int64  
dtypes: float64(3), int32(1), int64(3), object(6)
memory usage: 

In [107]:
data_pc_enviar = data_pc.loc[data_pc.id_distribuidor.isin(data_enviar.APPLICANT_ID)]

In [108]:
# colocando info sobre los ciclos

data_pc_enviar

,id_distribuidor,perdida_de_cosecha,inicio_credito,ciclo_credito
142,120016,0.0,2023-01-13,3
193,120023,0.0,2023-01-13,3
194,130978,0.0,2023-01-13,2
208,108748,0.0,2023-01-16,4
209,130077,0.0,2023-01-23,2
...,...,...,...,...
23981,172833,0.0,2023-12-28,1
23982,172834,0.0,2023-12-28,1
23983,172838,0.0,2023-12-28,1
23984,172839,0.0,2023-12-28,1


In [109]:
data_enviar#.APPLICANT_ID.nunique()

,APPLICANT_ID,ESCOLARIDAD,SUCURSAL,TIPO_NEGOCIO,TIEMPO_OP_NEGOCIO,COMPRAS,INGRESO_MENSUAL_NEGOCIO,VENTAS,NOMBREOTORGANTE,FOLIO,IMPORTECREDITO,FECHACONSULTA,SCORE_FICO
0,157169,TECNICO,GRUPO_4,VENTAS,40,1300.0,16000.0,4000.0,VOLANA,1186242652,1000,2023-07-04,618
1,157169,TECNICO,GRUPO_4,VENTAS,40,1300.0,16000.0,4000.0,MICROFINANCIERA,1186242652,1000,2023-06-21,618
2,157169,TECNICO,GRUPO_4,VENTAS,40,1300.0,16000.0,4000.0,SIC,1186242652,0,2022-12-29,618
3,157169,TECNICO,GRUPO_4,VENTAS,40,1300.0,16000.0,4000.0,SOCIEDAD FINANCIERA DE OBJETO MULTIPLE,1186242652,8000,2022-11-05,618
4,157169,TECNICO,GRUPO_4,VENTAS,40,1300.0,16000.0,4000.0,BANCO,1186242652,0,2022-12-29,618
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104778,157058,TECNICO,GRUPO_4,SERVICIOS,73,300.0,6032.0,1000.0,FINANCIERA,1252865143,0,2022-04-25,384
104779,157058,TECNICO,GRUPO_4,SERVICIOS,73,300.0,6032.0,1000.0,FINANCIERA,1252865143,0,2022-02-10,384
104780,157058,TECNICO,GRUPO_4,SERVICIOS,73,300.0,6032.0,1000.0,SERVICIOS,1252865143,0,2022-02-06,384
104781,157058,TECNICO,GRUPO_4,SERVICIOS,73,300.0,6032.0,1000.0,FINANCIERA,1252865143,0,2022-02-03,384


In [110]:
data_enviar.to_csv("./data/enviar/data_backtesting.csv", index=False)
data_pc_enviar.to_csv("./data/enviar/data_pc_backtesting.csv", index=False)

In [111]:
data_pc_enviar.id_distribuidor

142      120016
193      120023
194      130978
208      108748
209      130077
          ...  
23981    172833
23982    172834
23983    172838
23984    172839
23985    172840
Name: id_distribuidor, Length: 5937, dtype: int32

In [112]:
cnx.close()

In [64]:
!pip install dnspython

  Obtaining dependency information for dnspython from https://files.pythonhosted.org/packages/87/a1/8c5287991ddb8d3e4662f71356d9656d91ab3a36618c3dd11b280df0d255/dnspython-2.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   --- ------------------------------------ 30.7/307.7 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 92.2/307.7 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 194.6/307.7 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  307.2/307.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 1.5 MB/s eta 0:00:00


In [82]:
import smtplib
import dns.resolver

def verify_email_smtp(email):
    domain = email.split('@')[1]
    
    # Obtener el registro MX del dominio
    try:
        mx_records = dns.resolver.resolve(domain, 'MX')
        mx_record = str(mx_records[0].exchange)
    except Exception as e:
        return False
    
    # Conectar al servidor SMTP
    try:
        server = smtplib.SMTP()
        server.set_debuglevel(0)
        server.connect(mx_record)
        server.helo(server.local_hostname)
        server.mail('me@example.com')
        code, message = server.rcpt(email)
        server.quit()
        
        if code == 250:
            return True
        else:
            return False
    except Exception as e:
        return False

# Ejemplo de uso
email = "a01750443@tec.mx"
print(verify_email_smtp(email))  # Debería imprimir True si el correo electrónico es válido según el servidor SMTP


KeyboardInterrupt: 

In [67]:
domain

'pernexium.com'